In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True

In [2]:
#Inputs
# parcels_2019_shp = "C:\\Users\\slawless\\Desktop\\S.Projects 2021\\Parcel Splitting\\Code\\SplittingInputs.gdb\\Parcels_20200921Updated"
parcels_2019_shp = r".\weber.gdb\weber_parcels"

# building_points_shp = "C:\\Users\\slawless\\Desktop\\S.Projects 2021\\Parcel Splitting\\Code\\SplittingInputs.gdb\\AddressPoint"
# '.\\Inputs\\AddressPoints.gdb\\AddressPoints_Weber\\address_pts_lyr'
building_points_shp = r".\AddressPoints.gdb\AddressPoints_Weber"

#setting up output workspace gdb and environments- if it doesn't exist, create
env.outputCoordinateSystem = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')

if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "Parcel_splitting.gdb"]
gdb = os.path.join(outputs[0], outputs[1])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

## Create Mesh

In [3]:
desc = arcpy.Describe(parcels_2019_shp)

#create the acre mesh- in this case, the cell_width and height calculate to 1 acre.
acre_mesh = arcpy.management.CreateFishnet(out_feature_class = os.path.join(gdb,'acre_mesh'),
    origin_coord = str(desc.extent.lowerLeft),
    y_axis_coord = str(desc.extent.XMin) + " " + str(desc.extent.YMax + 10),
#     cell_width = 208.7, # feet
#     cell_height = 208.7, # feet
    cell_width = 63.6, # meters
    cell_height = 63.6, # meters
    number_rows = '#',
    number_columns = '#',
    labels = 'NO_LABELS', 
    template = parcels_2019_shp, 
    geometry_type = 'POLYGON'
)

# #Add Required Fields: Grid ID and RealBuildingID
arcpy.management.AddFields(in_table=acre_mesh, 
                           field_description=[["RealBldg", "DOUBLE", "RealBldg", "", "", ""]])

<Result '.\\Outputs\\Parcel_splitting.gdb\\acre_mesh'>

## Identify Building Points and Parcels that meet qualifications

In [4]:
#Identifying which parcels meet the qualifications
Parcels_7Acre_wBuildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_2019_shp, 
                                                                       out_path=gdb, out_name="Parcels_7Acre_wBuildings", 
                                                                       where_clause="building_type_id NOT IN (0, 14, 99, 3, 4, 5, 6, 8, 9, 10, 12) And PARCEL_ACRES >= 7")

#Identifying which buildings reside on those parcels that meet qualifications
Buildings_7Acre_wBuildings = arcpy.analysis.SpatialJoin(target_features=building_points_shp, 
                                                        join_features=Parcels_7Acre_wBuildings, 
                                                        out_feature_class=os.path.join(gdb,'Buildings_7Acre_wBuildings'), 
                                                        join_operation="JOIN_ONE_TO_ONE", 
                                                        join_type="KEEP_COMMON",
                                                        match_option="INTERSECT", 
                                                        search_radius="", 
                                                        distance_field_name="")

#Adding the Preserve Bldg field- used to identify which building areas to preserve
Buildings_7Acre_wBuildings = arcpy.management.AddField(in_table=Buildings_7Acre_wBuildings, 
                                                       field_name="PreserveBldg", 
                                                       field_type="SHORT", 
                                                       field_precision=None, 
                                                       field_scale=None, 
                                                       field_length=None, 
                                                       field_alias="RealBldg", 
                                                       field_is_nullable="NULLABLE", 
                                                       field_is_required="NON_REQUIRED", 
                                                       field_domain="")

Buildings_7Acre_wBuildings= arcpy.management.CalculateField(in_table=Buildings_7Acre_wBuildings, 
                                                    field="PreserveBldg", 
                                                    expression="1", 
                                                    expression_type="ARCADE", 
                                                    code_block="", 
                                                    field_type="TEXT")

Preserve_Buildings = arcpy.conversion.FeatureClassToFeatureClass(in_features=Buildings_7Acre_wBuildings, 
                                                                       out_path=gdb, out_name="Preserve_Buildings")

## Identify Mesh Pieces with Buildings

In [5]:
#Use Spatial Join to identify which acre_mesh pieces contain those preserve_buildings
Mesh_w_Buildings = arcpy.analysis.SpatialJoin(target_features=acre_mesh, 
                           join_features=Preserve_Buildings, 
                           out_feature_class=os.path.join(gdb,'Mesh_w_Buildings'), 
                           join_operation="JOIN_ONE_TO_ONE", 
                           join_type="KEEP_ALL",
                           match_option="INTERSECT", 
                           search_radius="", 
                           distance_field_name="")

#convert to new Feature Class if Preserve_blg = 1
Mesh_w_PreserveBldgs = arcpy.conversion.FeatureClassToFeatureClass(in_features=Mesh_w_Buildings, 
                                                                       out_path=gdb, out_name="Mesh_w_PreserveBldgs", 
                                                                       where_clause="PreserveBldg IN (1)")



## Select Parcels with Mesh Pieces

In [6]:
#Select the parcels with buildings on them and prep for mesh erasure and convert to feature class
Parcels_for_Erase_select = arcpy.management.SelectLayerByLocation(in_layer=[parcels_2019_shp],
                                                                 overlap_type="INTERSECT", 
                                                                 select_features=Buildings_7Acre_wBuildings, 
                                                                 search_distance="", 
                                                                 selection_type="NEW_SELECTION", 
                                                                 invert_spatial_relationship="NOT_INVERT")

Parcels_for_Erase= arcpy.conversion.FeatureClassToFeatureClass(in_features=Parcels_for_Erase_select, 
                                                                       out_path=gdb, out_name="Parcels_for_Erase")

## Erase Mesh Pieces from Selected Parcels

In [7]:
parcels_erased = arcpy.analysis.Erase(in_features=Parcels_for_Erase,
                                      erase_features=Mesh_w_PreserveBldgs, 
                                      out_feature_class= os.path.join(gdb,'parcels_erased'),
                                      cluster_tolerance="")



## Erase the building-related attributes from the parcels_erased

In [8]:
# #calculating all building parcel attributes to zero out for the non-building pieces
#YEAR BUILT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BUILT_YR", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

#BUILDING SQFT
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="BLDG_SQFT", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

# #NON-RESIDENTIAL-SQFT
# parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
#                                                                   field="non_residential_sqft", 
#                                                                   expression="""0""", 
#                                                                   expression_type="PYTHON3")

# #UNITS PER ACRE
# parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
#                                                                   field="UnitsperAc", 
#                                                                   expression="""0""", 
#                                                                   expression_type="PYTHON3")

#BUILDING TYPE ID
parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
                                                                  field="building_type_id", 
                                                                  expression="""0""", 
                                                                  expression_type="PYTHON3")

# #RESIDENTIAL UNITS
# parcels_erased = arcpy.management.CalculateField(in_table=parcels_erased, 
#                                                                   field="residential_units", 
#                                                                   expression="""0""", 
#                                                                   expression_type="PYTHON3")

## Get the "erased pieces" back and merge with with "erased parcels"

In [9]:
#join erased parcels to original parcels- puzzle piece 1
parcels_with_structures = arcpy.analysis.SymDiff(Parcels_for_Erase, parcels_erased, os.path.join(gdb,'parcels_with_structures'))
arcpy.AddField_management(parcels_with_structures, 'Split', 'LONG')
arcpy.CalculateField_management(parcels_with_structures, field='Split', expression=2, expression_type="PYTHON3") 

parcels_erased_union = arcpy.management.Merge(inputs=[parcels_erased, parcels_with_structures],
                                           output=os.path.join(gdb,'parcels_erased_union'),
                                           add_source="NO_SOURCE_INFO")

## Select all other parcel pieces

In [10]:
#All other parcels not included in the parcels_erased_union
Other_parcels_select = arcpy.management.SelectLayerByLocation(in_layer=[parcels_2019_shp], 
                                                          overlap_type="INTERSECT", 
                                                          select_features=Buildings_7Acre_wBuildings, 
                                                          search_distance="", 
                                                          selection_type="NEW_SELECTION", 
                                                          invert_spatial_relationship="INVERT")
#convert selection to new feature class
All_other_parcels= arcpy.conversion.FeatureClassToFeatureClass(in_features=Other_parcels_select, 
                                                                       out_path=gdb, 
                                                               out_name="All_other_parcels")


## Merge all parcels ready to split, back together

In [11]:
#merging the non-erased, and the erased parcels back together
prepared_parcels = arcpy.management.Merge(inputs=[All_other_parcels, parcels_erased_union],
                                           output=os.path.join(gdb,'prepared_parcels'),
                                           add_source="NO_SOURCE_INFO")


## All "need splitting parcels"

In [12]:
#select parcels with acreage larger than 7 or haven't been erased
parcels_YES_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause="PARCEL_ACRES >= 7 And (FID_parcels_erased > -1 Or FID_parcels_erased IS NULL)", 
                                                                               invert_where_clause="")

parcels_YES_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_YES_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="parcels_YES_split")

arcpy.CalculateField_management(parcels_YES_split, field='Split', expression=1, expression_type="PYTHON3") 

<Result '.\\Outputs\\Parcel_splitting.gdb\\parcels_YES_split'>

## Split selected parcels into 5 acre pieces

In [13]:
parcels_split = arcpy.management.SubdividePolygon(in_polygons=parcels_YES_split, 
                                  out_feature_class= os.path.join(gdb,'parcels_split'), 
                                  method="EQUAL_AREAS", 
                                  num_areas=None, 
                                  target_area="5 Acres", 
                                  target_width="", 
                                  split_angle=0, 
                                  subdivision_type="STACKED_BLOCKS")



## Select all Non-split parcels

In [14]:
#select all other parcels that arent ready for splitting
parcels_NO_split_select = arcpy.management.SelectLayerByAttribute(in_layer_or_view=prepared_parcels,
                                                                               selection_type="NEW_SELECTION", 
                                                                               where_clause="PARCEL_ACRES >= 7 And (FID_parcels_erased > -1 Or FID_parcels_erased IS NULL)", 
                                                                               invert_where_clause="INVERT")


parcels_NO_split = arcpy.conversion.FeatureClassToFeatureClass(in_features=parcels_NO_split_select, 
                                                                       out_path=gdb, 
                                                               out_name="parcels_NO_split")


## Put it all back together

In [15]:
#merging all parcels back together again
parcels_split_merge = arcpy.management.Merge(inputs=[parcels_split, parcels_NO_split],
                                           output=os.path.join(gdb,'parcels_split_merge'),
                                           add_source="NO_SOURCE_INFO")

# recalc acreage
arcpy.AddField_management(parcels_split_merge, 'PARCEL_ACRES_NEW', 'FLOAT')
arcpy.CalculateField_management(parcels_split_merge, "PARCEL_ACRES_NEW", """!SHAPE.area@ACRES!""", "PYTHON3")

<Result '.\\Outputs\\Parcel_splitting.gdb\\parcels_split_merge'>

## Attribute Cleanup

In [16]:
#attribute cleanup
final_parcel_df = pd.DataFrame.spatial.from_featureclass(parcels_split_merge)
#list(final_parcel_df.columns)

In [17]:
# divide land value by this number for new approximate value
final_parcel_df['Split_Factor'] = final_parcel_df['PARCEL_ACRES']/final_parcel_df['PARCEL_ACRES_NEW']
final_parcel_df['PARCEL_ACRES'] = final_parcel_df['PARCEL_ACRES_NEW']

fields = ['COUNTY_NAME', 'COUNTY_ID', 'PARCEL_ID', 'TOTAL_MKT_VALUE', 'LAND_MKT_VALUE', 'UNIT_COUNT', 'BLDG_SQFT', 'FLOORS_CNT',
         'BUILT_YR', 'EFFBUILT_YR', 'max_dua', 'max_far', 'max_height', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 
         'type7','type8', 'agriculture', 'basebldg', 'NoBuild', 'redev_friction', 'building_type_id', 'x', 'y', 
         'parcel_id_remm', 'PARCEL_ACRES', 'Split','Split_Factor', 'SHAPE']

# save old parcel id
final_parcel_df = final_parcel_df[fields].copy()
final_parcel_df['parcel_id_remm_old'] = final_parcel_df['parcel_id_remm']

# generate unique building id for split parcels
final_parcel_df.loc[(final_parcel_df['Split'].isna() == True), 'Split'] = 0
split_parcels = final_parcel_df[final_parcel_df['Split'] == 1].copy()
nonsplit_parcels = final_parcel_df[(final_parcel_df['Split']==0) | (final_parcel_df['Split']==2)].copy()
count = final_parcel_df.shape[0]
split_parcels['parcel_id_remm'] = count + split_parcels.index + 1

final_parcel_df = pd.concat([nonsplit_parcels, split_parcels])

# recalc land value and total market value using split factor
final_parcel_df.loc[(final_parcel_df['Split']==1), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']
final_parcel_df.loc[(final_parcel_df['Split']==1), 'TOTAL_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE']
final_parcel_df.loc[(final_parcel_df['Split']==2), 'TOTAL_MKT_VALUE'] = (final_parcel_df['TOTAL_MKT_VALUE'] - final_parcel_df['LAND_MKT_VALUE']) + (final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor'])
final_parcel_df.loc[(final_parcel_df['Split']==2), 'LAND_MKT_VALUE'] = final_parcel_df['LAND_MKT_VALUE'] / final_parcel_df['Split_Factor']

## Final export

In [18]:
final_parcel_df.spatial.to_featureclass(location=os.path.join(gdb,'parcels_split_FINAL'), sanitize_columns=False)

'E:\\Projects\\Parcel_Splitting\\Outputs\\Parcel_splitting.gdb\\parcels_split_FINAL'

In [19]:
# #opening the pro project to visualize
# import subprocess

# path_to_pro = 'C:\\Program Files\\ArcGIS\\Pro\\bin\\ArcGISPro.exe'
# path_to_file = 'C:\\Users\\slawless\\Desktop\\S.Projects 2021\\REMM Visualization\\Visualize Output Data- REMM\\Visualize Output Data- REMM.aprx'

# subprocess.call([path_to_pro, path_to_file])